----------

## <span style="color:green">Комментарий от наставника<span>

Привет еще раз! Спасибо, что быстро ответил. 

В этот раз пункты, где всё отлично, я отмечаю галкой <span style="color:green;font-size:200%">&#10003;</span>

Прошлые комментарии и твои ответы на них удалять не буду

Новые комментарии будут ниже, красным цветом. Вот так:

## <span style="color:red">Комментарий от наставника<span>

---------

# Задание 1  <span style="color:green;font-size:200%">&#10003;</span>
## Исправление данных  <span style="color:green;font-size:200%">&#10003;</span>

In [6]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem

credit_history = pd.read_csv('./datasets/data.csv')

##### Смотрим общую информацию по датасету. Видим пропущенные значения в days_employed и total_income

In [7]:
credit_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
Unnamed: 0          21525 non-null int64
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


###### Проверяем пересечение пропущенных значений 

In [8]:
missed_days_emloyed = len(credit_history[(credit_history['days_employed'].isnull())])
missed_total_income = len(credit_history[(credit_history['total_income'].isnull())])
missed_intersection = len(credit_history[(credit_history['days_employed'].isnull()) & (credit_history['total_income'].isnull())])

print('Пропущено значений days_employed:', missed_days_emloyed)
print('Пропущено значений missed_total_income:', missed_total_income)
print('Пересечение пропущенных значений:', missed_intersection)

Пропущено значений days_employed: 2174
Пропущено значений missed_total_income: 2174
Пересечение пропущенных значений: 2174


###### Ищем ошибки в данных children

In [9]:
credit_history.groupby('children').agg({'children': ['count'], 'days_employed': ['median'], 'total_income': ['median'], 'dob_years': ['median']})

,children,days_employed,total_income,dob_years
,count,median,median,median
children,,,,
-1,47,-1391.963378,144293.168218,41.0
0,14149,-1036.583486,143937.207982,48.0
1,4818,-1416.141843,147943.027779,38.0
2,2055,-1634.342227,144648.563117,35.0
3,330,-1681.245021,157221.549275,36.0
4,41,-1866.426406,156135.210434,35.0
5,9,-1231.571486,186351.410251,36.0
20,76,-957.251278,144544.530170,41.5


Встречаются системные ошибки связанные с вводом данных(-1 и 20 детей). 
> Рекомендация: Хорошо бы на этапе ввода данных сделать автоматическую проверку значений

Испавляем данные

In [10]:
credit_history['children'].value_counts()
credit_history.loc[credit_history['children'] == -1, 'children'] = 1
credit_history.loc[credit_history['children'] == 20, 'children'] = 2

###### Смотрим income_type

In [11]:
credit_history.groupby('income_type').agg({'children': ['count'], 'days_employed': ['median'], 'total_income': ['median'], 'dob_years': ['median']})

,children,days_employed,total_income,dob_years
,count,median,median,median
income_type,,,,
безработный,2,366413.652744,131339.751676,38.0
в декрете,1,-3296.759962,53829.130729,39.0
госслужащий,1459,-2689.368353,150447.935283,40.0
компаньон,5085,-1547.382223,172357.950966,39.0
пенсионер,3856,365213.306266,118514.486412,60.0
предприниматель,2,-520.848083,499163.144947,42.5
сотрудник,11119,-1574.202821,142594.396847,39.0
студент,1,-578.751554,98201.625314,22.0


Категории безработный, в декрете, преприниматель, студент встречаются крайне редко, их можно удалить

In [12]:
credit_history = credit_history[~credit_history['income_type'].isin(['безработный', 'в декрете', 'предприниматель', 'студент'])]

###### Смотрим на dob_years

In [13]:
credit_history.groupby('dob_years').agg({'children': ['count'], 'days_employed': ['median'], 'total_income': ['median'], 'dob_years': ['median']}).head()

,children,days_employed,total_income,dob_years
,count,median,median,median
dob_years,,,,
0,101,-1146.689586,152419.184851,0
19,14,-724.492610,93343.132708,19
20,51,-674.838979,109502.056040,20
21,111,-618.733817,128265.720871,21
22,182,-699.061214,124286.115523,22


Для 101 записи указан возраст 0, это ошибочное значение, заменим его на NaN

In [14]:
credit_history['dob_years'].value_counts().reset_index().sort_values(by='index')
credit_history.loc[credit_history['dob_years'] == 0, 'dob_years']  = np.nan
credit_history['dob_years'].value_counts().reset_index().sort_values(by='index').head()

,index,dob_years
53,19.0,14
51,20.0,51
46,21.0,111
44,22.0,182
41,23.0,254


##### Смотрим days_employed

In [15]:
credit_history.groupby('days_employed').agg({'children': ['count'], 'days_employed': ['median'], 'total_income': ['median'], 'dob_years': ['median']}).sample(10)

,children,days_employed,total_income,dob_years
,count,median,median,median
days_employed,,,,
-1374.398893,1,-1374.398893,37645.460469,47.0
-126.869645,1,-126.869645,313479.598835,47.0
-444.401628,1,-444.401628,189847.058085,57.0
-5645.284815,1,-5645.284815,161393.753158,37.0
-1593.158977,1,-1593.158977,139304.472779,39.0
-2192.155978,1,-2192.155978,185711.295160,27.0
-247.405024,1,-247.405024,147207.870612,28.0
-620.684193,1,-620.684193,118397.622433,34.0


Видим два типа ошибок:
* Отрицательный стаж. Переведём его в положительный
* Для безработных и пенсионеров слишком большой стаж(около 1000 лет). Если разница между возрастом стажем больше 14 лет, то удаляем стаж. Если стаж больше 100 лет, то тоже удаляем
> Рекомендация: поправить алгоритм расчёта стажа, есть смысл округлять стаж до года

Исправляем

In [16]:
credit_history['days_employed'] = credit_history['days_employed'].abs()
credit_history.groupby('income_type').agg({'days_employed': ['min', 'max']})
credit_history.loc[credit_history['days_employed'] / 365 - credit_history['dob_years'] > 14 , 'days_employed'] = np.nan
credit_history.loc[credit_history['days_employed'] / 365 > 100 , 'days_employed'] = np.nan

##### Смотрим education

In [17]:
credit_history.groupby('education').agg({'children': ['count'], 'days_employed': ['median'], 'total_income': ['median'], 'dob_years': ['median']})

,children,days_employed,total_income,dob_years
,count,median,median,median
education,,,,
ВЫСШЕЕ,274,1506.982306,167242.211789,40.0
Высшее,267,1605.977588,180186.682659,39.0
НАЧАЛЬНОЕ,17,1012.313022,129666.892242,46.0
НЕОКОНЧЕННОЕ ВЫСШЕЕ,29,949.418158,163911.559034,32.0
Начальное,15,906.489175,104570.429306,40.0
Неоконченное высшее,47,1178.018163,160402.480835,31.0
СРЕДНЕЕ,771,1730.411872,134337.067409,45.0
Среднее,711,1766.011755,135105.847892,45.0


Для последующего анализа поля education все значения надо перевести в нижний регистр. 
> Рекомендация: фиксировать список возможных значений. Вообще не понятно как с одними id появились разные education

In [18]:
credit_history['education'] = credit_history['education'].str.lower()

##### Смотрим family_status

In [19]:
credit_history.groupby('family_status').agg({'children': ['count'], 'days_employed': ['median'], 'total_income': ['median'], 'dob_years': ['median']})

,children,days_employed,total_income,dob_years
,count,median,median,median
family_status,,,,
Не женат / не замужем,2812,1234.662323,144682.347364,36.0
в разводе,1195,1693.774783,146969.347256,46.0
вдовец / вдова,960,2331.334484,128213.686950,58.0
гражданский брак,4175,1542.607468,144902.625471,41.0
женат / замужем,12377,1746.279695,146187.858312,43.0


Только в одном значении family_status "Не женат / не замужем" встречается заглавая буква. Надо перевести значения в нижний регистр. 
> Рекомендация: поправить справочник значений

In [20]:
credit_history['family_status'] = credit_history['family_status'].str.lower()

----------

## <span style="color:green">Комментарий от наставника<span>

Работа хорошая, но не структурираваная. Прошу внимательно прочитай "Инструкцию по выполнению" и обрати внимание к критериям оценки, в частности "Оставляет комментарии к шагам". Очень сложно для восприятия, когда сначала идет простыня с описанием дейтсвий а потом сами действия и результат. 

### Что нужно доработать:

1. Изучите общую информацию по датасету
> Добавил data.info()
1. Абсолютно не ясно как ты пришел к списку признаков, которые надо править
> Я группировал данные по каждому столбцу примерно так: 
> data.groupby('income_type').agg({'children': ['count'], 'days_employed': ['median'], 'total_income': ['median'], 'dob_years': ['median']}) 
> и смотрел выбросы. 
1. Нет описания и выводов по найденным проблемам в данных
> Добавил описание
1. Желательно имена переменных давать со смыслом. Название переменной data, к сожалению не отражает сути хранимых в нем данных
> Переименовал в credit_history
1. Перед удалением полезно посмотреть какой объём выборки затронет операция
> Посмотрел на предыдущем шаге

---------

----------

## <span style="color:red">Комментарий от наставника<span>

Еще раз прошу обраттиь внимание на структуру работы. Внимательно прочитай "Инструкцию по выполнению" и обрати внимание к критериям оценки, в частности "Оставляет комментарии к шагам". Очень сложно для восприятия, когда сначала идет простыня с описанием дейтсвий а потом сами действия и результат. 

Комментарии ты оставляешь в коде, гораздо информативнее они будут смотреться в markdown с должной структурой

> Разбил всё по шагам с markdown!

## Заполнение пропусков <span style="color:green;font-size:200%">&#10003;</span>

В столбцах days_emploed и total_income пропущены 2174 значений. Странно что пропуски в одних и тех же строках. Зависимости от других параметров не наблюдаю, объединяет эти два поля только тип данных: float64. Возможно при внесении данных использовался неправильный разделитель: "," вместо "."(или наоборот). Для анализа нас интересует поле total_income, заполним его медианным значением

In [21]:
# рассчитываем мединный доход в зависимости от типа дохода, образования и пола
credit_history_notnull_total_income = credit_history[credit_history['total_income'].notnull()].groupby(['income_type', 'education_id', 'gender']).agg({'total_income': 'median'})
credit_history_notnull_total_income.rename(columns={'total_income': 'total_income_median'}, inplace = True)

credit_history = credit_history.merge(credit_history_notnull_total_income, on=['income_type', 'education_id', 'gender'], how = 'left')
credit_history.loc[credit_history['total_income'].isnull(), 'total_income'] = credit_history['total_income_median']
credit_history.drop(columns=['total_income_median'], inplace = True)


# credit_history

# credit_history[credit_history['days_employed'].isnull()].groupby(['children', 'education_id', 'family_status_id', 'gender', 'income_type']).median()['days_employed']

----------

## <span style="color:green">Комментарий от наставника<span>

### Что нужно доработать:

Почему именно медианной заполнил пропуски?
> Доход относится к количественным переменным и может варьироваться в широком диапазоне. Чтобы избежать влияния отдельных высоких и низких зарплат выбрал медиану
---------

## Замена типов <span style="color:green;font-size:200%">&#10003;</span>
Доход можно привести к int

In [22]:
try:
    credit_history['total_income'] = credit_history['total_income'].astype('int')
except:
     print('преобразование не удалось')

----------

## <span style="color:green">Комментарий от наставника<span>

Отлично, хорошо, что использовал try, except
    
---------

## Поиск и удаление дубликатов <span style="color:green;font-size:200%">&#10003;</span>

In [23]:
print('Количество дубликатов ДО удаления:', credit_history.duplicated().sum())
credit_history = credit_history.drop_duplicates().reset_index(drop = True)
print('Количество дубликатов ПОСЛЕ удаления:', credit_history.duplicated().sum())
# credit_history

Количество дубликатов ДО удаления: 0
Количество дубликатов ПОСЛЕ удаления: 0


----------

## <span style="color:green">Комментарий от наставника<span>


### Что нужно доработать:

Перед удалением полезно посмотреть какой объём выборки затронет операция
> Добавил вывод

---------

## Выделение леммы  и группировка для Цели кредита <span style="color:green;font-size:200%">&#10003;</span>

После лемматизации получил список слов и частоту. Удалил предлоги и общие слова. Обобщил до категорий
* недвижимость       6349 --> недвижимость
* жилье              4458 --> недвижимость / ремонт
* автомобиль         4305 --> автомобиль
* образование        4013 --> образование
* операция           2604 --> недвижимость
* свадьба            2323 --> свадьба
* строительство      1877 --> недвижимость
* высокий            1374 --> образование
* коммерческий       1311 --> недвижимость
* жилой              1229 --> недвижимость
* дополнительный      906 --> образование
* заниматься          904 --> образование
* проведение          767 --> свадьба
* сдача               650 --> недвижимость
* семья               638 --> недвижимость
* собственный         634 --> недвижимость
* ремонт              606 --> ремонт
* подержанный         486 --> автомобиль
* подержать           478 --> автомобиль
* приобретение        461 --> автомобиль
* профильный          436 --> образование

#### На что берут кредит:
* недвижимость
* автомобиль
* образование
* свадьба
* ремонт

#### Добавляем в датасет цель кредита с помощью полученного словаря




In [24]:
m = Mystem()

def text_to_lemma(text):
    return m.lemmatize(text)

credit_history['purpose_lemma'] = credit_history['purpose'].apply(text_to_lemma)

words = credit_history['purpose_lemma'].apply(pd.Series)
words_list = words.values.ravel()
pd.value_counts(pd.Series(words_list))

# Фильтровал датасет по словам, проверял соответствие слов категориям
# credit_history[credit_history.purpose_lemma.apply(lambda x: 'ремонт' in x)]
# credit_history[credit_history.purpose_lemma.apply(lambda x: 'заниматься' in x)]
# credit_history[credit_history.purpose_lemma.apply(lambda x: 'высокий' in x)]
# credit_history[credit_history.purpose_lemma.apply(lambda x: 'профильный' in x)]

# добавляем группировку
def group_lemma(lemma):
    if 'ремонт' in lemma:
        return 'ремонт'
    elif 'недвижимость' in lemma:
        return 'недвижимость'
    elif 'жилье' in lemma:
        return 'недвижимость'
    elif 'автомобиль' in lemma:
        return 'автомобиль'
    elif 'образование' in lemma:
        return 'образование'
    elif 'операция' in lemma:
        return 'недвижимость'
    elif 'свадьба' in lemma:
        return 'свадьба'
    elif 'строительство' in lemma:
        return 'недвижимость'
    elif 'высокий' in lemma:
        return 'образование'
    elif 'коммерческий' in lemma:
        return 'недвижимость'
    elif 'жилой' in lemma:
        return 'недвижимость'
    elif 'дополнительный' in lemma:
        return 'образование'
    elif 'заниматься' in lemma:
        return 'образование'
    elif 'проведение' in lemma:
        return 'свадьба'
    elif 'сдача' in lemma:
        return 'недвижимость'
    elif 'семья' in lemma:
        return 'недвижимость'
    elif 'собственный' in lemma:
        return 'недвижимость'
    elif 'подержанный' in lemma:
        return 'автомобиль'
    elif 'подержать' in lemma:
        return 'автомобиль'
    elif 'приобретение' in lemma:
        return 'автомобиль'
    elif 'профильный' in lemma:
        return 'образование'

    return 'другое'

credit_history['purpose_group'] = credit_history['purpose_lemma'].apply(group_lemma)
credit_history.purpose_group.value_counts()
    


----------

## <span style="color:green">Комментарий от наставника<span>

### Что нужно доработать:

1. Когда пишешь код это действительно бывает удобно постпавить импорт в середину файла, но я б рекомоендовал убирать все импорты  в начало файла, подгружая таким образом все необходимые модули сразу 
> убрал в начало
1. Отличное решение. В данной задаче всего 5 категорий и ты решил задачу наиболее простым способом. А что делать если категорий больше?
> в общем случае можно прикрутить NLP, гуглил бы что-то типа такого https://towardsdatascience.com/text-classification-in-python-dd95d264c802
---------

----------

## <span style="color:red">Комментарий от наставника<span>

Применение NLP тут не целесообразно. Я имел ввиду более простое решение средствами Python. Подумай, но это задачка со звездочкой. Приму и без неё.

## !! Второй вариант категоризации
Выведем все леммы

In [0]:
credit_history['purpose_lemma'].value_counts()

* Составим справочник "лишних" слов и удалим их из массива. Можно было бы составить список "нужных" слов и удалить все остальные, но тогда мы могли бы удалить лишнее при обновлении датасета
* переименуем "жилье" в "недвижимость"


In [0]:
unnecessary_words = ['операция', 'покупка', 'проведение', 'сыграть', 'коммерческий', 'свой', 'для', 
                     'сдача', 'семья', 'собственный', 'строительство', 'жилой', 'высокий', 'заниматься', 'получение',
                    'дополнительный', 'подержанный', 'сделка', 'подержать', 'приобретение', 'профильный', 'ремонт',
                    ' ', '\n', 'на', 'с', 'со']

def purpose_del_unnecessary_words(purpose):
    purpose_proc = []
    for word in purpose:
        if word not in unnecessary_words:
            if word == 'жилье':
                word = 'недвижимость'
            purpose_proc.append(word)
    return purpose_proc

credit_history['purpose_lemma_category'] = credit_history['purpose_lemma'].apply(purpose_del_unnecessary_words)
credit_history['purpose_lemma_category'].value_counts()


Категория "ремонт" попала в "недвижимость". В первом варианте мы рассматривали "ремонт" на одном уровне с "недвижимостью" и "автомобилями", хотя "ремонт" - подкатегория категории "недвижимости".

# Задание2
## Есть ли зависимость между наличием детей и возвратом кредита в срок? <span style="color:green;font-size:200%">&#10003;</span>
##### Ответ: Да, есть
Сгруппируем данные по столбцу children. Если кол-во детей 0, то пишем 'нет детей', если больше 0, то 'есть дети'
У заёмщиков с детьми доля кредитов с просрочкой выше: 9,16% против 7,51%.
Можно посмотреть внимательнее на категорию с детьми, разделив их по количеству детей. Доля просрочки в категории 2+ детей выше чем в категории с одним ребёнком(9,25% и 9,14%). 

In [0]:
def children_group(children):
    if children == 0:
        return 'нет детей'
    else:
        return 'есть дети'
    
def debt_text(debt):
    if debt == 0:
        return 'нет просрочки'
    else:
        return 'есть просрочка'

def children_count_group(children):
    if children == 0:
        return 'нет детей'
    elif children == 1:
        return 'один ребёнок'
    else:
        return 'больше одного ребёнка'

credit_history['children_group'] = credit_history['children'].apply(children_group)
credit_history['debt_text'] = credit_history['debt'].apply(debt_text)
credit_history_children_group_debt = credit_history.pivot_table(index=['children_group'], columns = 'debt_text', values='children', aggfunc = 'count').reset_index()
credit_history_children_group_debt['% просрочки'] = credit_history_children_group_debt['есть просрочка'] / (credit_history_children_group_debt['есть просрочка'] + credit_history_children_group_debt['нет просрочки']) * 100
print(credit_history_children_group_debt.sort_values(by='% просрочки'))

credit_history['children_count_group'] = credit_history['children'].apply(children_count_group)
credit_history_children_debt = credit_history.pivot_table(index=['children_count_group'], columns = 'debt_text', values='debt', aggfunc = 'count').reset_index()
credit_history_children_debt['% просрочки'] = credit_history_children_debt['есть просрочка'] / (credit_history_children_debt['есть просрочка'] + credit_history_children_debt['нет просрочки']) * 100
print(credit_history_children_debt.sort_values(by='% просрочки'))



### Есть ли зависимость между семейным положением и возвратом кредита в срок? <span style="color:green;font-size:200%">&#10003;</span>
##### Ответ: Да, есть

Группируем данные по семейному статусу и считаем % просрочки как в предыдущем пункте. Видим что для вдовцов процент просрочки самый низкий, для не женатых самый высокий. Возникла гипотеза что определяющим является не семейный статус, а возраст. Чтобы это проверить разделим пользователей на возрастные группы по квантилям("19-33", "33-43", "43-53", "53+") и посмотрим есть ли зависимость по просрочке внутри этих возрастных групп. <br>
Как мы видим, внутри каждой возрастной группы разброс по семейному статусу существенный. Значит семейный статус действительно важен

In [0]:
credit_history_family_status_debt = credit_history.pivot_table(index=['family_status'], columns = 'debt_text', values='children', aggfunc = 'count').reset_index()
credit_history_family_status_debt
credit_history_family_status_debt['% просрочки'] = credit_history_family_status_debt['есть просрочка'] / (credit_history_family_status_debt['есть просрочка'] + credit_history_family_status_debt['нет просрочки']) * 100
print(credit_history_family_status_debt.sort_values(by='% просрочки'))

# Считаем квантили
# credit_history['dob_years'].quantile(q=0.25)
# credit_history['dob_years'].quantile(q=0.5)
# credit_history['dob_years'].quantile(q=0.75)

def dob_years_group(dob_years):
    if dob_years < 33:
        return '19 - 33'
    elif dob_years < 43:
        return '33 - 43'
    elif dob_years < 53:
        return '43 - 53'
    else:
        return '53+'
credit_history['dob_years_group'] = credit_history['dob_years'].apply(dob_years_group)
    
credit_history_family_status_debt = credit_history.pivot_table(index=['dob_years_group', 'family_status'], columns = 'debt_text', values='children', aggfunc = 'count').reset_index()
credit_history_family_status_debt
credit_history_family_status_debt['% просрочки'] = credit_history_family_status_debt['есть просрочка'] / (credit_history_family_status_debt['есть просрочка'] + credit_history_family_status_debt['нет просрочки']) * 100
print(credit_history_family_status_debt[['dob_years_group', 'family_status', '% просрочки']])

----------

## <span style="color:green">Комментарий от наставника<span>

Хорошая работа и интересные выводы

---------

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <span style="color:green;font-size:200%">&#10003;</span>

##### Ответ: Да, есть
Сгруппируем уровень дохода по квантиям и посмотрим на уровень просрочки внутри них. Самый низкий процент просрочки у самых обеспеченных заёмщиков - 7,06%. Далее идут самые бедные - 7,92%. "средний класс" платит хуже всех - 8,7%.

In [0]:
# Считаем квантили
# credit_history['total_income'].quantile(q=0.25)
# credit_history['total_income'].quantile(q=0.5)
# credit_history['total_income'].quantile(q=0.75)

def total_income_group(total_income):
    if total_income < 107516:
        return '0 - 107516'
    elif total_income < 144881:
        return '107516 - 144881'
    elif total_income < 196883:
        return '144881 - 196883'
    else:
        return '196883+'

credit_history['total_income_group'] = credit_history['total_income'].apply(total_income_group)

credit_history_total_income_group_debt = credit_history.pivot_table(index=['total_income_group'], columns = 'debt_text', values='children', aggfunc = 'count').reset_index()
credit_history_total_income_group_debt
credit_history_total_income_group_debt['% просрочки'] = credit_history_total_income_group_debt['есть просрочка'] / (credit_history_total_income_group_debt['есть просрочка'] + credit_history_total_income_group_debt['нет просрочки']) * 100
print(credit_history_total_income_group_debt.sort_values(by='% просрочки'))

### Как разные цели кредита влияют на его возврат в срок? <span style="color:green;font-size:200%">&#10003;</span>

Кредит на ремонт берут редко, но возвращают лучше всего. К кредитам по недвижимости заёмщики относятся ответственно. К кредитам на образование и автомобиль - нет.

In [0]:
credit_history_purpose_group_debt = credit_history.pivot_table(index=['purpose_group'], columns = 'debt_text', values='children', aggfunc = 'count').reset_index()
credit_history_purpose_group_debt
credit_history_purpose_group_debt['% просрочки'] = credit_history_purpose_group_debt['есть просрочка'] / (credit_history_purpose_group_debt['есть просрочка'] + credit_history_purpose_group_debt['нет просрочки']) * 100
print(credit_history_purpose_group_debt.sort_values(by='% просрочки'))

----------

## <span style="color:green">Комментарий от наставника<span>

Хорошее, удобное и понятное представление информации

### Что можно сделать лучше:

В твоих выводах нет гипотез о причинах таких зависимостей. Было бы полезно и интересно накидать гипотезы.
> Гипотезы:
> * Те, кто официально оформляют отношения, в целом более ответственные
> * Чем больше детей, тем выше постоянные расходы и ниже накопления. Заёмщик скорее просрочит кредит, чем сэкономит на детях
> * Если доход высокий, то есть накопления из-за этого вероятность просрочки ниже
> * Заёмщики с низкими доходами предпочитают занять у друзей и не быть должными банку, средний класс рассчитывает больше на себя



---------

----------

## <span style="color:red">Комментарий от наставника<span>

Супер!

### Вывод

Кредит - сложный продукт, на возврат которого влияет множество факторов. 
* Чем меньше детей, тем лучше заёмщик
* Те, кто были или состоят в официальном браке(женат, в разводе, вдовец) платят лучше, чем те, кто не оформляли отношения
* Богатые и бедные возвращают кредиты лучше, чем средний класс
* Каждый десятый кредит на автомобиль и образование выплачивается с просрочкой. Кредит на ремонт выплачивается лучше всего

##### Для более глубокого анализа не хватает данных по сумме кредита

----------

## <span style="color:green">Комментарий от наставника<span>

С почином! Ты молодец! В целом неплохая работа. Вот несколько выводов:

1. У тебя хороший и понятный код
1. Нужно поработать над структурой подачи материала. Разбить на шига и комментирова их. Это улучшитьт восприятие твоей работы
1. Нужно приводить промежуточные вычисления и цифры, даже если они очевидны для тебя. Твою работу может читать человек не так сильно погруженный в тему
1. Старайся записывать возможные гипотезы отсутствия или сомнения в данных, которые затем нужно перепроверить
1. Есть несколько предложений как можно улучшить работу
1. Во второй части работы есть инсайты и понятные хорошие выводы

**Доделай пункты отмеченные: "Что нужно доработать" и я жду твой проект**

---------

----------

## <span style="color:red">Комментарий от наставника<span>

**Прошу доработай струкутру в начале проекта**

----------

## <span style="color:green">Комментарий от наставника<span>

Поздравляю со сдачей первого проекта!
Отличная работа!

---------